CPSC 8810 Machine Learning for Biomedical Applications
# Practicum 01 - Data Preparation & Exploration

In this practicum, we will explore our first dataset, [Infrared Thermography Temperature](https://archive.ics.uci.edu/dataset/925/infrared+thermography+temperature+dataset). Per the UCI website, _"this dataset contains temperatures read from various locations of inferred images about patients, with the addition of oral temperatures measured for each individual. The 33 features consist of gender, age, ethnicity, ambiant temperature, humidity, distance, and other temperature readings from the thermal images"_. 


Here we will implement some of the concepts discussed in the previsous lecture, _Data Exploration and Preparation_. Specifically we:
- Visually inspect the distribution of continuous varialbes using Kernel Density Estimates (or Box Plots)
- Test continuous data for Noramlity
- Assess missing data 
- Assess feature correlation
- Develop a Table One
- Implement methods for continuous data normalization
- Implement methods for creating dummy variables for nominal features 

Later in the course, we will use this dataset to build regression models that will predict the temperature using the environment and the thermal image features. 

In [ ]:
# imports
from ucimlrepo import fetch_ucirepo 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro
import pandas as pd
from tableone import TableOne
from sklearn import preprocessing
import numpy as np 

# local project imports
import sys
sys.path.append('../../src') # alternatively add to path using: pip install -e /path/to/src
from plotting import plt_kde_grid, plt_box_grid
from uci_utils import get_vars_of_type

In [ ]:
# global settings
pd.options.display.max_columns = 100
# set threshold for normality tests
alpha = 0.01

# Retreive the data
Many of the UCI Machine Learning Repository datasets are configured to enable retreival using the `ucimlrepo` Python library. Here, we obtain the dataset and assign some of its elements to local variables.

In [ ]:
# fetch Infrared Thermography Dataset
# fetch dataset 
infrared_thermography_temperature = fetch_ucirepo(id=925) 
  
# data (as pandas dataframes) 
X = infrared_thermography_temperature.data.features 
y = infrared_thermography_temperature.data.targets 
subject_ids = infrared_thermography_temperature.data.ids
meta = infrared_thermography_temperature.metadata
meta_vars = infrared_thermography_temperature.variables
print(meta_vars[['name', 'description']])

# Data Exploration
Now, let's get to know our feature and target data a little better. 

## Target Data
First, let's examine the target variables:
1. aveOralF - Oral temperature measured in fast mode
2. aveOralM - Oral temperature measured in monitor mode

We will:
1. Check if there missing values for any samples
2. Plot the distributions of the two target variables using Kernel Density Estimates (KDE)
3. Test for normality 

First, let's check for missing values. We can use the Python Pandas `isna` and `sum` methods to find the number of rows with missing data for each column.

In [ ]:
y.isna().sum()

As we expect, there are no missing data in the targets. 

Now let's examine the KDEs of the target variables (which are continuous valued)

In [ ]:
fig, ax = plt_kde_grid(y, num_cols = 2, fig_size = (10, 5))

We can see that the distributions have long tails and, therefore, are probabaly not quite normally distributed. We can test this with the Shapiro-Wilk test (scipy docs with references, [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html)). We will store the names of variables for which the test indicates we should reject the null hypothesis (i.e., that the variable is normally distributed) to use later in creating our table one.

In [ ]:
non_normal_vars = []
for c in y.columns:
    _, p_value = shapiro(y[c])
    print(f'p-value for {c} = {p_value}')
    if p_value < alpha:
        non_normal_vars.append(c)

Indeed, the p-values returned by the Shapiro-Wilk test are well below any reasonable &alpha; threshold for rejecting the null hypothesis (typically 0.05 or 0.01), suggesting that we should consider these variables to be __not__ normally distributed. The implication is that in reporting on these data, we should prefer the median over the mean as a measure of centrality and the interquartile range (IQR) over the standard deviation as a measure of dispersiion. 

## Feature Data
Now let's examine the feature data. For the features that are continuous, we will plot the plot the distributions and test for normality as we did with the targtes. However, some of the features are nominal (categorical) for which we will prefer to view counts of each category.

First, let's count the number of rows with missing data from each column.

<span style ="color:dodgerblue">
<h2>Problem 1 (1 point)</h2> 
In the code cell below display the number of rows with missing data for each column of the feature data, <pre style="display:inline">X</pre>.</span>

In [ ]:
#### YOUR CODE HERE ####

We should see that the `X.Distance` variable has two missing values. Let's take a look at these two rows in the cell block below. Later, we will cover imputation methods to deal with missing values before training our ML models.

In [ ]:
X.loc[pd.isna(X['Distance']),:]

### Continuous Features
Now let's take a closer look at the continuous features. In the code cells below:
1. Plot the KDE of each continuous variable.
2. Print the result of the Shapiro-Wilk test and add any variables with a p-value &le; &alpha; to the `non_normal_vars` list
3. Examine the Pearson cross-correlation matrix.

<span style ="color:dodgerblue">
<h3>Problem 2 (1 point):</h3>
In the code cell below use the <pre style="display:inline">plt_kde_grid</pre> function to plot the continuous features, <pre style="display:inline">X_continuous</pre>, to get a sense of the variable value distributions</span>

In [ ]:
continuous_vars, X_continuous = get_vars_of_type(X, meta_vars, var_type_key = 'type', var_name_key = 'name', type_kw = 'Continuous')

#### YOUR CODE HERE ####

<span style ="color:dodgerblue">
<h3>Problem 3 (1 point): </h3>
In the code cell below use the `shapiro` function to apply the Shapiro-Wilk test of normality to the continuous features, `X_continuous`, and store the feature names of any feature with a p-value < &alpha; in the `non_normal_vars` list</span>

In [ ]:
#### YOUR CODE HERE ####

It is often to examine the linear cross-correlation of feature variables to see if they carry redundant signal. We will see later, that we may wish to keep only one feature among highly correlated features.

In [ ]:
# plot the cross-correlation matrix of the continuous variables. Set the colorbar to be between -1 and 1
fig, ax = plt.subplots(figsize = (10, 10))
sns.heatmap(X_continuous.corr(), vmin = -1, vmax = 1, cmap = 'coolwarm', ax = ax)
plt.show()

### Categorical Features

For the categorical features, we want to compute the counts and percents of the labels within each category. First let's get the categorical features.

<span style ="color:dodgerblue">
<h3>Problem 4 (1 point):</h3>
In the code cell below, display the counts and percents of samples with each of the possible labels for each nominal feature. For example, the `Gender` variable in this dataset is either `Female` or `Male` for each sample. We want to know the number and percent of samples that are `Female` and `Male`. HINT: consider using the `pandas.value_counts` method</span>


In [ ]:
categorical_vars, X_categorical = get_vars_of_type(X, meta_vars, var_type_key = 'type', var_name_key = 'name', type_kw = 'Categorical')

#### YOUR CODE HERE ####

# Table One
We can now put everything together in a Table One. This can be done manually using all the information we've developed above. Fortunately, this can also be produced using the [tableone package](https://tableone.readthedocs.io/en/latest/index.html).

In [ ]:
mytable = TableOne(X, categorical = categorical_vars, nonnormal=non_normal_vars)
print(mytable.tabulate(tablefmt="fancy_grid"))
mytable.to_csv('../../data/outputs/tableone.csv')

# Data Normalization

## Continuous Variables
Empirical evidence suggests that most machine learning models perform better when continuous variables are standardized, i.e., transformed to have zero mean and unit variance, or scaled so that all variables fall within the same range, e.g. $\left[-1,1\right]$. This is particularly important when the features are on differenent sclales. For example, notice above, that the mean value of the `Distance` variable is $O\left(0.1\right)$ while the rest of the variables are $O\left(10\right)$. The choice of standardizing vs. scaling may be dictated by the machine learning algorithm, some of which assume normality of the data. So even when the data is not strictly normal, we often ignore this and use standardized data when training a machine learning model.

Fortunately, there are libraries that include will allow us to easily scale the data. We will use [scikit-learn](https://scikit-learn.org/stable/index.html), which is amongst the most popular machine learning libraries. Later in the course, we will use these capabilities when we develop and evaluate our models.

First let's try standardizing the features. __Notice we are using the Pandas `dropna` method to remove the two rows with missing data.__

In [ ]:
scaler = preprocessing.StandardScaler().fit(X_continuous.dropna())
X_continuous_scaled = scaler.transform(X_continuous.dropna())

Now, let's look at the mean and standard deviation values of `X_continuous`. As we will see later, in supervised learning, we will divide our data in training and testing sets. When developing a mahcine learning model we always want to __avoid allowing test data characteristics to _leak_ into the training data__. This implies that we should compute the mean and standard deviation using only the training data and then scale the test data using the mean and standard deviation found for the training data. 

In [ ]:
# view the mean values of each column in the data
display(scaler.mean_)

# view the standard devatiation of each column in the data
display(scaler.scale_)

# check that the means are 0 and the standard deviations are 1 in the scaled data
display(X_continuous_scaled.mean(axis = 0).round(2))
display(X_continuous_scaled.std(axis = 0).round(2))

# view the first 5 rows of the scaled data. Notice that the scaler returns a numpy array, not a pandas dataframe, so we need to convert it to a dataframe
# if we want to see the column names
display(pd.DataFrame(X_continuous_scaled, columns=X_continuous.columns).head())

Now, let's try scaling the continuous features instead of standardizing. We will scale the data to be in $\left[-1,1\right]$ using scikit-learn's `MinMaxScaler`. In this case, the transformation of a data array, `X` is given by:

```X_std = (X-X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))```

```X_scaled = X_std * (max-min) + min```

<span style ="color:dodgerblue">
<h3>Problem 5 (1 point):</h3>
In the code cell below, use the `preprocessing.MinMaxScaler` from `scikit-learn` to scale the features in `X_continuous` to be in $\left[-1, 1\right]$. Then display the minimum of each column, the maximum of each column, and the scaled data in a Pandas DataFrame.</span>

In [ ]:
#### YOUR CODE HERE ####

## Nominal Features

Unlike continuous valued features, we do not standardize or scale categorical features. Most ofen, we will create dummy variables to replace the original categorical variable. For a given categorical variable, $x$, with $N$ possible values, we will replace $x$ with $N$ new values. Each of the new variables are binary valued. The value of the $n^{th}$ variable is set to 1 if $x$ is in the $n^{th}$ category and set to 0 otherwise.

For example, the `Gender` varialbe in this dataset can take the possible values `Female` and `Male`. To create dummy variables, we remove the `Gender` variable and create two new variables `Gender_male` and `Gender_female`. For a given sample, if the `Gender` variable is `Female`, the `Gender_female` is set to 1 and the `Gender_male` variable is set to 0. Otherwise, if the `Gender` variable is `Male`, the `Gender_female` is set to 0 and the `Gender_male` variable is set to 1. 

Fortunately, the Pandas `get_dummies` method will do most of the work needed to create dummy variables. Let's see how to implement this below. 

In [ ]:
# create dummy variables for the Gender variable
X_categorical_dummy = pd.get_dummies(X_categorical, columns=categorical_vars,drop_first=False, dtype=int)
display(X_categorical_dummy.head())

As a check on the dummy variable conversion, we can compute the percent of rows that have a value of 1 for each of the category types. These should match the percents we found for our Table 1.

<span style ="color:dodgerblue">
<h3>Problem 6 (1 point):</h3> 
In the code cell below, display the percentage of samples with each of the possible labels by summing each column in <pre style="display:inline">X_categorical_dummy</pre> and dividing by the total number of rows. For example, the percent of samples that are of age <pre style="display:inline">[21,25]</pre> is the sum of entries in the column <pre style="display:inline">Age_21-25</pre>.</span>

In [ ]:
#### YOUR CODE HERE ####

Finally, we can concatenate the dummy variables with the standardized continuous variables into a new single dataframe. This dataframe will represent the final version of the data we would use to train and test a machine learning model.

In [ ]:
#Create a new dataframe that contains the scaled continuous variables and the dummy variables
X_new = pd.concat([pd.DataFrame(X_continuous_scaled, columns=X_continuous.columns), X_categorical_dummy.reset_index()], axis=1)
display(X_new.head())